# Lab 03 - Polynomial Fitting
In the previous lab we discussed linear regression and the OLS estimator for solving the minimization of the RSS. As we
mentioned, regression problems are a very wide family of settings and algorithms which we use to try to estimate the relation between a set of explanatory variables and a **continuous** response (i.e. $\mathcal{Y}\in\mathbb{R}^p$). In the following lab we will discuss one such setting called "Polynomial Fitting". 

Sometimes, the data (and the relation between the explanatory variables and response) can be described by some polynomial
of some degree. Here, we only focus on the case where it is a polynomial of a single variable. That is: 
$$ p_k\left(x\right)=\sum_{i=0}^{k}\alpha_i x^i\quad\alpha_0,\ldots,\alpha_k\in\mathbb{R} $$

So our hypothesis class is of the form:
$$ \mathcal{H}^k_{poly}=\left\{p_k\,\Big|\, p_k\left(x\right)=\sum_{i=0}^{k}\alpha_i x^i\quad\alpha_0,\ldots,\alpha_k\in\mathbb{R}\right\} $$

Notice that similar to linear regression, each hypothesis in the class is defined by a coefficients vector. Below are two
examples (simulated and real) for datasets where the relation between the explanatory variable and response is polynomial.

In [9]:
import sys
sys.path.append("../")
from utils import *

In [10]:
response = lambda x: x**4 - 2*x**3 - .5*x**2 + 1

x = np.linspace(-1.2, 2, 40)[0::2]
y_ = response(x)

df = pd.read_csv("../datasets/Position_Salaries.csv", skiprows=2, index_col=0)
x2, y2 = df.index, df.Salary

make_subplots(1, 2, subplot_titles=(r"$\text{Simulated Data: }y=x^4-2x^3-0.5x^2+1$", r"$\text{Positions Salary}$"))\
    .add_traces([go.Scatter(x=x, y=y_, mode="markers", marker=dict(color="black", opacity=.7), showlegend=False),
                 go.Scatter(x=x2, y=y2, mode="markers",marker=dict(color="black", opacity=.7), showlegend=False)], 
                rows=[1,1], cols=[1,2])\
    .update_layout(title=r"$\text{(1) Datasets For Polynomial Fitting}$", margin=dict(t=100)).show()



As we have discussed in class, solving a polynomial fitting problem can be done by first manipulating the input data,
such that we represent each sample $x_i\in\mathbb{R}$ as a vector $\mathbf{x}_i=\left(x^0,x^1,\ldots,x^k\right)$. Then,
we treat the data as a design matrix $\mathbf{X}\in\mathbb{R}^{m\times k}$ of a linear regression problem.

For the simulated dataset above, which is of a polynomial of degree 4, the design matrix looks as follows: 


In [11]:
from sklearn.preprocessing import PolynomialFeatures
m, k, X = 5, 4, x.reshape(-1, 1)
pd.DataFrame(PolynomialFeatures(k).fit_transform(X[:m]), 
             columns=[rf"$x^{{0}}$".format(i) for i in range(0, k+1)],
             index=[rf"$x_{{0}}$".format(i) for i in range(1, m+1)])

,$x^0$,$x^1$,$x^2$,$x^3$,$x^4$
$x_1$,1.0,-1.200000,1.440000,-1.728000,2.073600
$x_2$,1.0,-1.035897,1.073083,-1.111604,1.151508
$x_3$,1.0,-0.871795,0.760026,-0.662587,0.577640
$x_4$,1.0,-0.707692,0.500828,-0.354432,0.250829
$x_5$,1.0,-0.543590,0.295490,-0.160625,0.087314


## Fitting A Polynomial Of Different Degrees

Next, let us fit polynomials of different degrees and different noise properties to study how it influences the learned model.
We begin with the noise-less case where we fit for different values of $k$. As we increase $k$ we manage to fit a model
that describes the data in a better way, reflected by the decrease in the MSE.

*Notice that in both the `PolynomialFeatures` and `LinearRegression` functions we can add the bias/intercept parameter. As in this case it makes no difference, we will include the bias in the polynomial features transformation and fit a linear regression **without** an intercept. The bias parameter of the polynomial features (i.e. $x^0$) will in reality be the intercept of the linear regression.*

In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

ks = [2, 3, 4, 5]
fig = make_subplots(1, 4, subplot_titles=list(ks))
for i, k in enumerate(ks):
    y_hat = make_pipeline(PolynomialFeatures(k), LinearRegression(fit_intercept=False)).fit(X, y_).predict(X)
    
    fig.add_traces([go.Scatter(x=x, y=y_, mode="markers", name="Real Points",  marker=dict(color="black", opacity=.7), showlegend=False),
                    go.Scatter(x=x, y=y_hat, mode="markers", name="Predicted Points",  marker=dict(color="blue", opacity=.7), showlegend=False)], rows=1, cols=i+1)
    fig["layout"]["annotations"][i]["text"] = rf"$k={{0}}, MSE={{1}}$".format(k, round(np.mean((y_-y_hat)**2), 2))

fig.update_layout(title=r"$\text{(2) Simulated Data - Fitting Polynomials of Different Degrees}$",
                  margin=dict(t=60),
                  yaxis_title=r"$\widehat{y}$",
                  height=300).show()

Once we find the right $k$ (which in our case is 4) we managed to fit a perfect model, after which, as we increase $k$, 
the additional coefficients will be zero.

In [13]:
coefs = {}
for k in ks:
    fit = make_pipeline(PolynomialFeatures(k), LinearRegression(fit_intercept=False)).fit(X, y_)
    coefs[rf"$k={{{k}}}$"] = [round(c,3) for c in fit.steps[1][1].coef_]
pd.DataFrame.from_dict(coefs, orient='index', columns=[rf"$w_{{{i}}}$" for i in range(max(ks)+1)])




,$w_{0}$,$w_{1}$,$w_{2}$,$w_{3}$,$w_{4}$,$w_{5}$
$k={2}$,0.971,-2.142,0.403,NaN,NaN,NaN
$k={3}$,0.672,-1.454,1.010,-0.564,NaN,NaN
$k={4}$,1.000,0.000,-0.500,-2.000,1.0,NaN
$k={5}$,1.000,0.000,-0.500,-2.000,1.0,0.0


## Fitting Polynomial Of Different Degrees - With Sample Noise

Still fitting for different values of $k$, let us add some standard Gaussian noise (i.e. $\mathcal{N}\left(0,1\right)$).
This time we observe two things:
- Even for the correct $k=4$ model we are not able to achieve zero MSE.
- As we increase $4<k\rightarrow 7$ we manage to decrease the error more and more.

In [14]:
y = y_ + np.random.normal(size=len(y_))

ks = range(2, 8)
fig = make_subplots(2, 3, subplot_titles=list(ks))
for i, k in enumerate(ks):
    r,c = i//3+1, i%3+1 
    
    y_hat = make_pipeline(PolynomialFeatures(k), LinearRegression(fit_intercept=False)).fit(X, y).predict(X)
    fig.add_traces([go.Scatter(x=x, y=y_, mode="markers", name="Real Points",  marker=dict(color="black", opacity=.7), showlegend=False),
                    go.Scatter(x=x, y=y, mode="markers", name="Observed Points",  marker=dict(color="red", opacity=.7), showlegend=False), 
                    go.Scatter(x=x, y=y_hat, mode="markers", name="Predicted Points",  marker=dict(color="blue", opacity=.7), showlegend=False)], rows=r, cols=c)
    fig["layout"]["annotations"][i]["text"] = rf"$k={{0}}, MSE={{1}}$".format(k, round(np.mean((y-y_hat)**2), 2))

fig.update_layout(title=r"$\text{(4) Simulated Data With Noise - Fitting Polynomials of Different Degrees}$", margin=dict(t=80)).show()

How is it that we are able to fit "better" models for $k$s larger than the true one? As we increase $k$ we enable the model
more "degrees of freedom" to try and adapt itself to the observed data. The higher $k$ the more the learner will "go after
the noise" and miss the real signal of the data. In other words, what we have just observed is what is known as **overfitting**.

Later in the course we will learn methods for detection and avoidance of overfitting.


## Fitting Polynomial Over Different Sample Noise Levels

Next, let us set $k=4$ (the true values) and study the outputted models when training over different noise levels. Though
we will only be changing the scale of the noise (i.e. the variance, $\sigma^2$), changing other properties such as its
distribution is interesting too. As we would expect, as we increase the scale of the noise our error increases. We can
observe this also in a visual manner, where the fitted polynomial (in blue) less and less resembles the actual model (in black).

In [15]:
scales = range(6)
fig = make_subplots(2, 3, subplot_titles=list(map(str, scales)))
for i, s in enumerate(scales):
    r,c = i//3+1, i%3+1
    
    y = y_ + np.random.normal(scale=s, size=len(y_))
    y_hat = make_pipeline(PolynomialFeatures(4), LinearRegression(fit_intercept=False)).fit(X, y).predict(X)

    fig.add_traces([go.Scatter(x=x, y=y_, mode="markers", name="Real Points",  marker=dict(color="black", opacity=.7), showlegend=False),
                    go.Scatter(x=x, y=y, mode="markers", name="Observed Points",  marker=dict(color="red", opacity=.7), showlegend=False),
                    go.Scatter(x=x, y=y_hat, mode="markers", name="Predicted Points",  marker=dict(color="blue", opacity=.7), showlegend=False)], rows=r, cols=c)
    fig["layout"]["annotations"][i]["text"] = rf"$\sigma^2={{0}}, MSE={{1}}$".format(s, round(np.mean((y-y_hat)**2), 2))

fig.update_layout(title=r"$\text{(5) Simulated Data - Different Noise Scales}$", margin=dict(t=80)).show()

## The Influence Of $k$ And $\sigma^2$ On Error

Lastly, let us check how the error is influenced by both $k$ and $\sigma^2$. For each value of $k$ and $\sigma^2$ we will
add noise drawn from $\mathcal{N}\left(0,\sigma^2\right)$ and then, based on the noisy data, let the learner select an
hypothesis from $\mathcal{H}_{poly}^k$. We repeat the process for each set of $\left(k,\sigma^2\right)$ 10 times and report
the mean MSE value. Results are seen in heatmap below: 


In [16]:
from sklearn.model_selection import ParameterGrid

df = []
for setting in ParameterGrid(dict(k=range(10), s=np.linspace(0, 5, 10), repetition=range(10))):
    y = y_ + np.random.normal(scale=setting["s"], size=len(y_))
    y_hat = make_pipeline(PolynomialFeatures(setting["k"]), LinearRegression(fit_intercept=False)).fit(X, y).predict(X)
    
    df.append([setting["k"], setting["s"], np.mean((y-y_hat)**2)])
    
df = pd.DataFrame.from_records(df, columns=["k", "sigma","mse"]).groupby(["k","sigma"]).mean().reset_index()

go.Figure(go.Heatmap(x=df.k, y=df.sigma, z=df.mse, colorscale="amp"),
          layout=go.Layout(title=r"$\text{(6) Average Train } MSE \text{ As Function of } \left(k,\sigma^2\right)$", 
                           xaxis_title=r"$k$ - Fitted Polynomial Degree",
                           yaxis_title=r"$\sigma^2$ - Noise Levels")).show()


# Time To Think...

In the above figure, we observe the following trends:
- As already seen before, for the noise-free data, once we reach the correct $k$ we achieve zero MSE.
- Across all values of $k$, as we increase $\sigma^2$ we get higher MSE values.
- For all noise levels, we manage to reduce MSE values by increasing $k$.

So, by choosing a **richer** hypothesis class (i.e. larger and that can express more functions - polynomials of higher
degree) we are able to choose an hypothesis that fits the **observed** data **better**, regardless to how noisy the data is.
Try and think how the above heatmap would look if instead of calculating the MSE over the training samples (i.e train error)
we would have calculated it over a **new** set of test samples drawn from the same distribution.

Use the below code to create a test set. Change the code generating figure 6 such that the reported error is a test error. Do not forget to add the noise (that depends on $\sigma^2$) to the test data. What has changed between what we observe for the train error to the test error? What happens for high/low values of $\sigma^2$? What happens for high/low values of $k$?

In [7]:
# Generate the x values of the test set
testX = np.linspace(-1.2, 2, 40)[1::2].reshape(-1,1)

# Generate the noisy y values of the test set. Set the noise level (the scale parameter) according to the specific setting
testY = response(testX) + np.random.normal(scale=___, size=len(y_))

ValueError: could not convert string to float: 

In [17]:
from sklearn.model_selection import ParameterGrid

df = []
for setting in ParameterGrid(dict(k=range(10), s=np.linspace(0, 5, 10), repetition=range(10))):
    y = y_ + np.random.normal(scale=setting["s"], size=len(y_))
    testY = response(testX) + np.random.normal(scale=setting["s"], size=len(testX))
    y_hat = make_pipeline(PolynomialFeatures(setting["k"]), LinearRegression(fit_intercept=False)).fit(X, y).predict(testX)

    df.append([setting["k"], setting["s"], np.mean((testY-y_hat)**2)])

df = pd.DataFrame.from_records(df, columns=["k", "sigma","mse"]).groupby(["k","sigma"]).mean().reset_index()

go.Figure(go.Heatmap(x=df.k, y=df.sigma, z=df.mse, colorscale="amp"),
          layout=go.Layout(title=r"$\text{(6) Average Train } MSE \text{ As Function of } \left(k,\sigma^2\right)$",
                           xaxis_title=r"$k$ - Fitted Polynomial Degree",
                           yaxis_title=r"$\sigma^2$ - Noise Levels")).show()
